In [1]:
from classifiers import CoTrainingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import pickle

## Load Models

### TF-IDF

In [2]:
with open ("tfidf.pk", 'rb') as fin:
    tfidf = pickle.load(fin)

In [3]:
tfidf

TfidfVectorizer(max_df=5, ngram_range=(1, 3), stop_words='english')

### Word2Vec CBOW

In [4]:
cbow = Word2Vec.load("cbow.model")

In [5]:
cbow

### Word2Vec SkipGram

In [6]:
skg = Word2Vec.load("skg.model")

In [7]:
skg

## Importing Data

In [8]:
df = pd.read_csv("processed_train.csv")

In [9]:
df

,Unnamed: 0,id,tweet,subtask_a,subtask_b,subtask_c
0,0,86426,"['ask', 'native', 'american', 'take']",OFF,UNT,NaN
1,1,90194,"['go', 'home', '’', 'drunk', 'maga', 'trump', ...",OFF,TIN,IND
2,2,16820,"['amazon', 'investigating', 'chinese', 'employ...",NOT,NaN,NaN
3,3,62688,"['someone', 'shouldve', 'taken', 'piece', 'shi...",OFF,UNT,NaN
4,4,43605,"['obama', 'wanted', 'liberal', 'amp', 'illegal...",NOT,NaN,NaN
...,...,...,...,...,...,...
13235,13235,95338,"['sometimes', 'get', 'strong', 'vibe', 'people...",OFF,TIN,IND
13236,13236,67210,"['benidorm', '✅', 'creamfields', '✅', 'maga', ...",NOT,NaN,NaN
13237,13237,82921,"['report', 'garbage', 'dont', 'give', 'crap']",OFF,TIN,OTH
13238,13238,27429,['pussy'],OFF,UNT,NaN


### Removing Redundant Axes

In [10]:
df = df.drop(['Unnamed: 0', 'subtask_b', 'subtask_c', 'id'], axis=1)

In [11]:
df

,tweet,subtask_a
0,"['ask', 'native', 'american', 'take']",OFF
1,"['go', 'home', '’', 'drunk', 'maga', 'trump', ...",OFF
2,"['amazon', 'investigating', 'chinese', 'employ...",NOT
3,"['someone', 'shouldve', 'taken', 'piece', 'shi...",OFF
4,"['obama', 'wanted', 'liberal', 'amp', 'illegal...",NOT
...,...,...
13235,"['sometimes', 'get', 'strong', 'vibe', 'people...",OFF
13236,"['benidorm', '✅', 'creamfields', '✅', 'maga', ...",NOT
13237,"['report', 'garbage', 'dont', 'give', 'crap']",OFF
13238,['pussy'],OFF


### Renaming Columns

In [12]:
df = df.rename(columns={'subtask_a': 'Offensive'})

In [13]:
df

,tweet,Offensive
0,"['ask', 'native', 'american', 'take']",OFF
1,"['go', 'home', '’', 'drunk', 'maga', 'trump', ...",OFF
2,"['amazon', 'investigating', 'chinese', 'employ...",NOT
3,"['someone', 'shouldve', 'taken', 'piece', 'shi...",OFF
4,"['obama', 'wanted', 'liberal', 'amp', 'illegal...",NOT
...,...,...
13235,"['sometimes', 'get', 'strong', 'vibe', 'people...",OFF
13236,"['benidorm', '✅', 'creamfields', '✅', 'maga', ...",NOT
13237,"['report', 'garbage', 'dont', 'give', 'crap']",OFF
13238,['pussy'],OFF


### Converting Offensive to Numerical Value

In [14]:
def off(cls):
    if cls =='OFF':
        return 1
    return 0

In [15]:
df["Offensive"] = df["Offensive"].apply(off)

In [16]:
df

,tweet,Offensive
0,"['ask', 'native', 'american', 'take']",1
1,"['go', 'home', '’', 'drunk', 'maga', 'trump', ...",1
2,"['amazon', 'investigating', 'chinese', 'employ...",0
3,"['someone', 'shouldve', 'taken', 'piece', 'shi...",1
4,"['obama', 'wanted', 'liberal', 'amp', 'illegal...",0
...,...,...
13235,"['sometimes', 'get', 'strong', 'vibe', 'people...",1
13236,"['benidorm', '✅', 'creamfields', '✅', 'maga', ...",0
13237,"['report', 'garbage', 'dont', 'give', 'crap']",1
13238,['pussy'],1


In [17]:
df['tweet']

0                    ['ask', 'native', 'american', 'take']
1        ['go', 'home', '’', 'drunk', 'maga', 'trump', ...
2        ['amazon', 'investigating', 'chinese', 'employ...
3        ['someone', 'shouldve', 'taken', 'piece', 'shi...
4        ['obama', 'wanted', 'liberal', 'amp', 'illegal...
                               ...                        
13235    ['sometimes', 'get', 'strong', 'vibe', 'people...
13236    ['benidorm', '✅', 'creamfields', '✅', 'maga', ...
13237        ['report', 'garbage', 'dont', 'give', 'crap']
13238                                            ['pussy']
13239    ['spanishrevenge', 'v', 'justice', 'human', 'r...
Name: tweet, Length: 13240, dtype: object

## Fitting Word Models on Data

In [18]:
X_train, X_test, y_train, y_test = train_test_split(df['tweet'], df['Offensive'], stratify=df['Offensive'], random_state=0)

In [19]:
X_train

3759             ['friend', 'father', 'safe', 'talk', '”']
1623                                   ['president', 'im']
8650     ['depending', 'government', 'politician', 'fur...
7897     ['im', 'sure', 'really', 'low', 'carb', 'diet'...
365                   ['welcome', 'why', 'james', 'cryin']
                               ...                        
11365    ['nra', 'supported', 'gun', 'control', 'reagan...
8409     ['quite', 'good', 'faking', 'must', 'done', 'n...
7883                                             ['thank']
10443    ['abundantly', 'clear', 'lack', 'common', 'sen...
10972                 ['there', 'brexit', '👇', '🏻', 'url']
Name: tweet, Length: 9930, dtype: object

### TF-IDF

In [43]:
X_train_tf = tfidf.transform(X_train.sort_index())
print(X_train_tf)

  (0, 90335)	0.5384430828382877
  (0, 8544)	0.5958519306604624
  (0, 8543)	0.5958519306604624
  (1, 61615)	0.5
  (1, 61614)	0.5
  (1, 39779)	0.5
  (1, 39778)	0.5
  (2, 145820)	0.31916827891130445
  (2, 145819)	0.31916827891130445
  (2, 109653)	0.2884172110522567
  (2, 93482)	0.31916827891130445
  (2, 93481)	0.31916827891130445
  (2, 75129)	0.31916827891130445
  (2, 64184)	0.31916827891130445
  (2, 64183)	0.31916827891130445
  (2, 4902)	0.31916827891130445
  (2, 4901)	0.31916827891130445
  (3, 75838)	0.7071067811865476
  (3, 71905)	0.7071067811865476
  (4, 141686)	0.44398266100968137
  (4, 131741)	0.41907298524281483
  (4, 120388)	0.4637545829882441
  (4, 78480)	0.44398266100968137
  (4, 67452)	0.4637545829882441
  (5, 63427)	0.7071067811865476
  :	:
  (9929, 123888)	0.1765599555837432
  (9929, 123887)	0.1765599555837432
  (9929, 123883)	0.16903241883392575
  (9929, 123882)	0.16903241883392575
  (9929, 123881)	0.3273830930277344
  (9929, 119405)	0.1765599555837432
  (9929, 119403)	0.169

In [34]:
X_test_tf = tfidf.transform(X_test.sort_index())

### Sentence Vectoriser

Finding the average vector for a given document

In [22]:
def sent_vect(sent, model):
    sent = eval(sent)
    vec = np.zeros(32)
    num = 0
    for w in sent:
        try:
            temp_vec = model.wv.get_vector(w)
            #print(temp_vec)
            vec = np.add(vec, temp_vec)
            num += 1
        except:
            pass
        
    return vec / np.sqrt(vec.dot(vec))

### CBOW

In [23]:
X_train_cbow = X_train.apply(sent_vect, model=cbow).sort_index()

In [24]:
X_train_cbow

0        [0.11678230874736859, 0.04701858112529256, 0.1...
1        [0.023887964134799274, -0.04221032997250825, 0...
4        [0.023089552048990755, -0.036891745280832305, ...
5        [0.08365669444110241, 0.024382291167563585, 0....
7        [0.1692174577116111, -0.028866439854166506, 0....
                               ...                        
13232    [0.05910713569141994, 0.03033006832724931, 0.2...
13235    [0.1772960903524207, 0.06816052902725968, 0.19...
13236    [-0.07846772515660165, -0.09172202196583056, 0...
13237    [0.1720133490472096, 0.0603833914465248, 0.166...
13239    [0.029200375665037092, -0.027191909181750278, ...
Name: tweet, Length: 9930, dtype: object

In [35]:
X_test_cbow = X_test.apply(sent_vect, model=cbow).sort_index()

In [36]:
X_test_cbow

2        [-0.05144699518391731, -0.07000303002084465, 0...
3        [0.10839988236478272, -0.02007087490801532, 0....
6        [0.14215383139067395, 0.09612889083374788, 0.0...
9        [0.14834794895835923, 0.041729775171655985, 0....
23       [0.10737067888646858, 0.06788053575314568, 0.0...
                               ...                        
13224    [0.07261746754903796, 0.023324642599656067, 0....
13229    [0.11278004817370701, 0.06896479115835308, 0.1...
13233    [0.11422348815947328, -0.03982285937728821, 0....
13234    [-0.04389888644976187, -0.1105844745764699, 0....
13238    [0.06189782589780631, 0.05593079023601799, 0.1...
Name: tweet, Length: 3310, dtype: object

### SkipGram

In [37]:
X_train_skg = X_train.apply(sent_vect, model=skg).sort_index()

In [38]:
X_train_skg

0        [0.13890833028577879, 0.0602093866169931, 0.25...
1        [-0.031044015373555757, -0.06391622386404926, ...
4        [0.009941511388440752, 0.006697370287529366, 0...
5        [0.05088327548790825, 0.12449198214735306, 0.2...
7        [0.20541999370674277, -0.011201584641294629, 0...
                               ...                        
13232    [0.0820706311814619, 0.05471222033757831, 0.22...
13235    [0.10886295323000372, 0.06637772834474372, 0.1...
13236    [-0.04250946890486125, -0.005367833791252949, ...
13237    [0.18586203624193126, 0.07148610685130281, 0.2...
13239    [0.02059650501444647, -0.010558274514424206, 0...
Name: tweet, Length: 9930, dtype: object

In [39]:
X_test_skg = X_test.apply(sent_vect, model=skg).sort_index()

In [40]:
X_train_skg

0        [0.13890833028577879, 0.0602093866169931, 0.25...
1        [-0.031044015373555757, -0.06391622386404926, ...
4        [0.009941511388440752, 0.006697370287529366, 0...
5        [0.05088327548790825, 0.12449198214735306, 0.2...
7        [0.20541999370674277, -0.011201584641294629, 0...
                               ...                        
13232    [0.0820706311814619, 0.05471222033757831, 0.22...
13235    [0.10886295323000372, 0.06637772834474372, 0.1...
13236    [-0.04250946890486125, -0.005367833791252949, ...
13237    [0.18586203624193126, 0.07148610685130281, 0.2...
13239    [0.02059650501444647, -0.010558274514424206, 0...
Name: tweet, Length: 9930, dtype: object

## Co - Training

### SVM vs SVM

In [41]:
svm2 = CoTrainingClassifier(SVC())

In [42]:
svm2.fit(X_train_tf, X_train_cbow, y_train)

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Int64Index([   2,    3,    6,    9,   23,\n            ...\n            9913, 9916, 9923, 9925, 9926],\n           dtype='int64', length=2482). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"